# 30-05-24 Tenta
## Uppgift 1

In [1]:
from mtm026 import *

EI, L, q0 = sp.symbols('EI L q0') 

K = sp.zeros(6, 6)
K1 = Ke_balk(EI, L)
K2 = K1
assem(K, K1, dofs=[1,2,3,4])
assem(K, K2, dofs=[3,4,5,6])

displayvar('K', K)

<IPython.core.display.Math object>

In [2]:
f = sp.zeros(6, 1)
f1 = fe_balk(q=-q0, L=L) + fe_balk_linjär_last(q=q0, L=L)
f2 = fe_balk_linjär_last(q=-q0, L=L)
assem(f, f1, dofs=[1,2,3,4])
assem(f, f2, dofs=[3,4,5,6])

displayvar('f', f)

<IPython.core.display.Math object>

In [3]:
a1, a2, a3, a4, a5, a6 = sp.symbols('a1:7')
r1, r2, r3, r4, r5, r6 = sp.symbols('r1:7')

a = sp.Matrix([0, 0, a3, a4, 0, 0])
r = sp.Matrix([r1, r2, 0, 0, r5, r6])

eq = sp.Eq(K*a, r + f)
unkowns = [r1, r2, a3, a4, r5, r6]

sol = sp.solve(eq, unkowns)

display(sol)

{a3: -7*L**4*q0/(240*EI),
 a4: 0,
 r1: L*q0/2,
 r2: 5*L**2*q0/24,
 r5: L*q0/2,
 r6: -5*L**2*q0/24}

In [4]:
a = a.subs(sol)
r = r.subs(sol)

displayvar('a', a)
displayvar('r', r)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Uppgift 2

In [5]:
EI, L, P = 1, 1, 1

K = np.zeros([7, 7])
K_sig = np.zeros([7, 7])

ke1 = Ke_balk(EI, L)
ke2 = ke1
ke3 = Ke_fjäder(k=EI/L**3)
assem(K, ke1, dofs=[1,2,3,4])
assem(K, ke2, dofs=[3,4,5,6])
assem(K, ke3, dofs=[3, 7])

ke1_sig = Ke_sigma_balk(P, L)
ke2_sig = Ke_sigma_balk(P, L)
assem(K_sig, ke1_sig, dofs=[1,2,3,4])
assem(K_sig, ke2_sig, dofs=[3,4,5,6])

displayvar('K', K)
displayvar('K_\sigma', K_sig)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
free_dofs = [3, 4, 6]

a = sp.symbols('a3 a4 a6')
K_red = extract_block(K, free_dofs, free_dofs)
K_sig_red = extract_block(K_sig, free_dofs, free_dofs)

displayvar('K_{red}', K_red)
displayvar('K_{\sigma, red}', K_sig_red)
displayvar('a_{red}', a)

# a, svar

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
# Lös egenvärdesproblemet (K - α K_σ)n = 0
# Behöver inte skriva om till ett vanligt egenvärdesproblem för det finns lösare för det 
# generaliserade egenvärdesproblemet 

alpha, knäckningsmoder = eigh(K_red, K_sig_red) # => egenvektorer & egenvärden 
# vill man enbart ha realdelen till α kan man skriva α.real 

# Skriv ut lastmultiplikatorerna 
print(f"α = {[f'{val.real:.2f}' for val in alpha]}")
print(f"Kristisk last P_kr = {alpha.real[0]*P:.2f} N")

# b, svar

α = ['5.53', '18.85', '49.38']
Kristisk last P_kr = 5.53 N


In [8]:
print('\n', knäckningsmoder)

# c, svar


 [[ 0.59455474  0.27654721 -0.02412169]
 [ 0.42837351 -1.20235267 -1.49861409]
 [-1.27950166  1.36019526 -2.12432209]]


## Uppgift 3

In [9]:
S = sp.Matrix([
    [-120, 80, 0],
    [80, 100, 40],
    [0, 40, -120]
              ])

eigenvals = S.eigenvals()

displayvar('Huvudspänningar', eigenvals)


<IPython.core.display.Math object>

In [10]:
sigma1 = -10 * np.sqrt(201) -10
sigma2 = -120
sigma3 = -10 +10*np.sqrt(201)

print('Huvudspänningar: ', [sigma1, sigma2, sigma3])
print('Högsta drag', f'{sigma3:.2f} MPa')
print('Högsta tryck', f'{sigma1:.2f} MPa')

# a, svar

Huvudspänningar:  [-151.77446878757826, -120, 131.77446878757826]
Högsta drag 131.77 MPa
Högsta tryck -151.77 MPa


In [11]:
von_m = 1 / np.sqrt(2) * np.sqrt((sigma1 - sigma2)**2 + (sigma2 - sigma3)**2 + (sigma3 - sigma1)**2)

print('Effektivspänning: ', von_m, 'MPa')

# b, svar

Effektivspänning:  269.07248094147417 MPa


In [12]:
n = sp.Matrix([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)])

s = S * n
sigma = n.T * s  # Detta är en 1x1-matris
sigma = sigma[0]  # Omvandla till skalär
tao = sp.sqrt(s.norm()**2 - sigma**2)

# Visa resultaten
print(f"Normalspänning σ_n: {sigma:.2f}")
print(f"Skjuvspänning τ: {np.abs(tao):.2f}")

# c, svar

Normalspänning σ_n: 33.33
Skjuvspänning τ: 133.00


## Uppgift 4

In [13]:
d = 150e-3      #m
D = 300e-3      #m
p0 = 180e6      #Pa
E = 200e9       #Pa
v = 0.3         #-

# RV
# sigma_r(d/2) = -p0
# sigma_r(D/2) = 0

a = np.array([[1, -1/(d/2)**2], [1, -1/(D/2)**2]])
b = np.array([-p0, 0])
AB = np.linalg.solve(a,b)
A = AB[0]
B = AB[1]

def sigma_phi(r):
    return A + B / r**2

sigma_phi_max = sigma_phi(d/2)
displayvar('\sigma_\phi', sigma_phi_max)

# a, svar

<IPython.core.display.Math object>

In [14]:
D1 = 450e-3             # m
sigma_phi_goal = 200e6  # Pa

sigma_phi_new = sigma_phi_goal - sigma_phi_max

# sigma_phi(d/2) = -100e6
# sigma_r(D/2) = 0
a = np.array([[1, 1/(d/2)**2], [1, -1/(d/2)**2]])
b = np.array([-100e6, 0])
AB = np.linalg.solve(a,b)
A = AB[0]
B = AB[1]


# sigma_r (D/2) = -q
def sigma_r(r):
    return A - B / r**2

def delta(q, D, d, d0):
    return q * 2*d**3*(D**2-d0**2) / (E * (D**2-d**2)*(d**2-d0**2))

q = -sigma_r(D/2)

print(q*10**-6, 'MPa')
print(delta(q, D1, D, d)*10**3, 'mm')

37.5 MPa
0.23999999999999996 mm


## Uppgift 5

In [15]:
h = 10e-3           # m
R2 = 14e-3          # m
R = R2/2            # m
a0 = 4e-3           # m
sigma_min = 0       # Pa
sigma_max = 150e6   # Pa
sigma_s = 620e6     # Pa
K_Ic = 36           # MNm-3/2
n = 4
C = 11.2*10**-12    # m/(MNm-3/2)**n

def f3(a):
    return np.sqrt((a + R)/a)
def K_I(sigma0, a):
    return sigma0 * np.sqrt(np.pi * a) * f3(a)


# Brott fås då K_I >= K_Ic
# Utgå från att sigma0 = sigma_max

a_c = 0.0001    # Avoid division by 0

while (K_I(sigma_max, a_c) < K_Ic * 10**6):
    a_c += 0.0001

a_c *= 10**3    # mm

ASTM = 2.5 * (K_Ic*10**6 / sigma_s)**2 *10**3   # mm

print("Simulation method gives: [mm]")
displayvar('a_c', a_c)
displayvar('ASTM', ASTM)

# a, svar

Simulation method gives: [mm]


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
# Paris law
# da / dN = C(∆K)**n

a = sp.symbols("a")

f = sp.sqrt((a+R)/a)

sigma_max *= 10**-6     # Why?
ΔK = sigma_max * sp.sqrt(np.pi*a) * f

I = 1 / (C * ΔK**n)

a_c *= 10**-3
N = sp.integrate(I, (a, a0, a_c))

displayvar('N', N)

# b, svar

<IPython.core.display.Math object>